In [ ]:
#Plotting variation in Predicted and Actual Values
import matplotlib.pyplot as plt
aY = range(10)
Y_Test_plot=Y_test[:10]
Predictor_plot=predictor[:10]
# Plotting variation in Predicted and Actual Values
plt.figure(figsize=(8, 5))
plt.plot(aY, Predictor_plot, marker='o', linestyle='-', color='b', label='Predicted Values')  # Line 1
plt.plot(aY, Y_Test_plot, marker='s', linestyle='--', color='r', label='Actual Values')  # Line 2


plt.xlabel('Index')
plt.ylabel('Values')
plt.title('Variation in Predicted and Actual Values')
plt.legend()
plt.tight_layout()

plt.show()